In [1]:
from datetime import *
from gmplot import gmplot
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime', 'time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
weather=pd.read_csv('austin_weather.csv').drop(['DewPointHighF','DewPointLowF','HumidityHighPercent','HumidityLowPercent',
                                               'SeaLevelPressureHighInches','SeaLevelPressureLowInches','VisibilityHighMiles',
                                               'VisibilityLowMiles','WindHighMPH','WindGustMPH','HumidityAvgPercent'],1)
trips=pd.read_csv('austin-bike/austin_bikeshare_trips.csv')
stations = pd.read_csv('austin_bikeshare_stations_w_zipcode.csv').drop(['latitude','longitude','status','name','Unnamed: 0'],1)

weather['Rain']=np.where(weather.Events.str.contains('Rain'), 1, 0)
weather.Date=pd.to_datetime(weather.Date)
weather=weather.set_index('Date').drop('Events',1)

ind = pd.DatetimeIndex(trips.start_time)
trips['Date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour
by_date = trips.pivot_table('trip_id', aggfunc='count',index='Date',columns='subscriber_type', )
by_date = trips.groupby(['Date', 'subscriber_type'])['trip_id'].count()
by_date.name = 'count'
by_date = by_date.reset_index('subscriber_type')
trips_2=trips[trips.year.isin([2014,2015,2016])]
trip_weather=trips_2.merge(weather,left_on='Date',right_on='Date',
                         suffixes=['_trips','_weather']).drop(['SeaLevelPressureAvgInches'],1)

trip_weather["start_time"] = trip_weather["start_time"].map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_weather["hours"] = trip_weather["start_time"].map(lambda x : x.hour)
val=''
def time_of_day(x):
    global val
    if (x>=6)&(x<10):
        val='morning rush hours'
    elif (x>=15)&(x<19):
        val='afternoon ruch hours'
    elif (x>=10)&(x<15):
        val='noon'
    elif (x>=19)|(x<6):
        val='night'
    return val
trip_weather['Time of the day'] = trip_weather['hours'].apply(lambda a: time_of_day(a))

trip_zipcode=trip_weather.merge(stations,left_on='start_station_id',right_on='station_id')

trip_zipcode.loc[trip_zipcode['zipcode'] != 78705, 'University_vicinity'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'University_vicinity'] = 1
trip_zipcode.head()

holidays = ['2014-01-01', '2015-01-01', '2016-01-01','2014-01-18', '2014-01-19','2014-01-20','2015-01-17', 
            '2015-01-18','2015-01-19','2016-01-16', '2016-01-17','2016-01-18','2014-02-15', '2014-02-16', '2014-02-17',
            '2015-02-14', '2015-02-15', '2015-02-16','2016-02-13', '2016-02-14', '2016-02-15',
            '2014-05-24', '2014-05-25', '2014-05-26','2015-05-23', '2015-05-24', '2015-05-25','2016-05-28', '2016-05-29', 
            '2016-05-30','2014-07-04', '2014-07-05', '2014-07-06','2015-07-03', '2015-07-04', '2015-07-05',
            '2016-07-02', '2016-07-03', '2016-07-04','2014-08-30', '2014-08-31', '2014-09-01','2015-09-05', '2015-09-06',
            '2015-09-07','2016-09-03', '2016-09-04', '2016-09-05','2014-10-11', '2014-10-12', '2014-10-13',
            '2015-10-10', '2015-10-11', '2015-10-12','2016-10-08', '2016-10-09', '2016-10-10','2014-11-11', '2015-11-11',
            '2016-11-11','2014-11-27', '2014-11-28', '2014-11-29', '2014-11-30','2015-11-26', '2015-11-27', '2015-11-28',
            '2015-11-29','2016-11-24', '2016-11-25', '2016-11-26', '2016-11-27','2014-12-25', '2014-12-26', '2014-12-27',
            '2014-12-28','2015-12-25', '2015-12-26', '2015-12-27', '2015-12-28','2016-12-24', '2016-12-25', '2016-12-26',
            '2016-12-27','2014-03-07', '2014-03-16','2015-03-13', '2015-03-22', '2016-12-27','2014-03-07', '2014-03-16',
            '2015-03-13', '2015-03-22','2016-03-11', '2016-03-20','2014-10-03', '2014-10-05','2014-10-10', '2014-10-12',
            '2015-10-02', '2015-10-04','2015-10-09', '2015-10-11','2016-09-30', '2016-10-02','2016-10-07', '2016-10-09']
trip_zipcode.loc[-trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 0
trip_zipcode.loc[trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 1
trip_zipcode.head()

trip_zipcode['weekday'] = trip_zipcode['Date'].apply(lambda x: x.weekday())
trip_zipcode.loc[trip_zipcode['weekday'].isin([1,2,3,4,5]), 'Weekday_flag'] = 1
trip_zipcode.loc[trip_zipcode['weekday'].isin([6,7]), 'Weekday_flag'] = 0
trip_zipcode.head()

trip_zipcode=trip_zipcode.drop(['end_station_id','end_station_name','start_station_name','TempHighF','TempLowF','DewPointAvgF',
                  'VisibilityAvgMiles','hours','location','station_id','PrecipitationSumInches',
                  'start_time','subscriber_type','year','hour','WindAvgMPH'],1)

In [4]:
trips_2 = pd.read_csv("austin-bike/austin_bikeshare_trips.csv")

In [5]:
trips = trip_zipcode
trips["start_time"] = trips_2["start_time"]
trips["end_station_id"] = trips_2["end_station_id"]
#transfer time into datetime
trips["start_time"] = trips["start_time"].map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trips["hours"] = trips["start_time"].map(lambda x : x.hour)
trips["weekday"] = trips["start_time"].map(lambda x : x.isoweekday())

In [6]:
end = datetime.datetime.strptime("2017", "%Y")
start = datetime.datetime.strptime("2014", "%Y")
trips_t_control = trips[(trips["start_time"] > start) & (trips["start_time"] <= end)]

In [7]:
def hours_to_category(hours):
    if hours >= 7 and hours <= 9:
        return "rush_m"
    elif hours >= 10 and hours <= 4:
        return "day time"
    elif hours >=5 and hours <= 7:
        return "rush_n"
    else:
        return "others"

In [8]:
trips["time_cat"] = trips["hours"].map(hours_to_category)

trips_t_control["time_cat"] = trips_t_control["hours"].map(hours_to_category)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#stations_close_McCombs = stations[(stations["latitude"] < 30.30) & (stations["latitude"] > 30.27) \
#                                  & (stations["longitude"] < -97.6) & (stations["longitude"] > -97.8)\
#                                 & (stations["status"] == "active")]
#stations_close_McCombs

In [ ]:
#borrow_heatmap_df = trips_t_control[["start_station_id","time_cat"]].merge(stations_close_McCombs[["station_id", "latitude", "longitude","location"]],
#                                                                           left_on = "start_station_id",
#                                                                          right_on = "station_id")
#borrow_heatmap_df[:5]

In [18]:
stations = pd.read_csv('austin-bike/austin_bikeshare_stations.csv')

Mask
1. Monday
2. rush_morning


In [19]:
def create_plotting_data(mask):
    masked_df = trips[mask]
    borrowed_df = pd.DataFrame(masked_df.groupby("start_station_id")["start_station_id"].count())
    borrowed_df["station_id"] = borrowed_df.index.values
    returned_df = pd.DataFrame(masked_df.groupby("end_station_id")["end_station_id"].count())
    returned_df["station_id"] = returned_df.index.values

    circle_df_borrow = borrowed_df.merge(stations[["station_id", "latitude", "longitude","location"]],
                                                                              left_on = "station_id",
                                                                              right_on = "station_id")
    circle_df_borrow.columns = ["bikes_borrowed", "start_station_id", "latitude", "longitude","location"]

    circle_df_return = returned_df.merge(stations[["station_id", "latitude", "longitude","location"]],
                                                                              left_on = "station_id",
                                                                              right_on = "station_id")
    circle_df_return.columns = ["bikes_returned", "end_station_id", "latitude", "longitude","location"]

    circle_df = circle_df_borrow.merge(circle_df_return, left_on = "start_station_id",
                                      right_on = "end_station_id", how = "outer")
    return circle_df

In [64]:
def plot_circles(circle_df, plot_name ,radius_expand):
    gmap = gmplot.GoogleMapPlotter(30.28398,-97.73792, 13)

    for i in range(0, circle_df.shape[0]):
        if not (np.isnan(circle_df["latitude_x"].iloc[i]) or np.isnan(circle_df["latitude_y"].iloc[i]) ):
            if circle_df["bikes_borrowed"].iloc[i] > circle_df["bikes_returned"].iloc[i]:
                gmap.circle(circle_df["latitude_y"].iloc[i] , 
                         circle_df["longitude_y"].iloc[i], 
                         radius = circle_df["bikes_returned"].iloc[i] *radius_expand,
                         color = "blue"
                        )
                gmap.circle(circle_df["latitude_x"].iloc[i] , 
                         circle_df["longitude_x"].iloc[i], 
                         radius = circle_df["bikes_borrowed"].iloc[i]*radius_expand,
                        color = "red"
                        )
#                print 1,circle_df["latitude_x"].iloc[i], circle_df["longitude_x"].iloc[i],circle_df["bikes_borrowed"].iloc[i]*radius_expand
            else:
                gmap.circle(circle_df["latitude_x"].iloc[i] , 
                         circle_df["longitude_x"].iloc[i], 
                         radius = circle_df["bikes_borrowed"].iloc[i]*radius_expand,
                        color = "red"
                        )
                gmap.circle(circle_df["latitude_y"].iloc[i] , 
                         circle_df["longitude_y"].iloc[i], 
                         radius = circle_df["bikes_returned"].iloc[i]*radius_expand,
                         color = "blue"
                        )
#                print 0, circle_df["latitude_x"].iloc[i], circle_df["longitude_x"].iloc[i],circle_df["bikes_borrowed"].iloc[i]*radius_expand
    gmap.draw(plot_name)

In [65]:
In_Genral_m = [True] * len(trips)
df = create_plotting_data(In_Genral_m)
plot_circles(df, "In_Genral.html", 0.01)

In [66]:
rush_m_sunny = ((trips["time_cat"] == "rush_m") & (trips["Rain"] == 0) & (trips["weekday"] < 6))
df_1 = create_plotting_data(rush_m_sunny )
plot_circles(df_1, "Morning_Rush_Hours_Sunny.html", 0.1)

In [67]:
rush_m_rainy = ((trips["time_cat"] == "rush_m") & (trips["Rain"] == 1) & (trips["weekday"] < 6))
df_2 = create_plotting_data(rush_m_rainy )
plot_circles(df_2, "Morning_Rush_Hours_Rainy.html", 0.1)

In [69]:
rush_m_sunny = ((trips["time_cat"] == "rush_n") & (trips["Rain"] == 0) & (trips["weekday"] < 6))
df_3 = create_plotting_data(rush_m_sunny )
plot_circles(df_3, "Afternoon_Rush_Hours_Sunny.html", 1)

In [72]:
rush_m_rain = ((trips["time_cat"] == "rush_n") & (trips["Rain"] == 1) & (trips["weekday"] < 6))
df_4 = create_plotting_data(rush_m_rain )
plot_circles(df_4, "Afternoon_Rush_Hours_Rain.html", 1)

In [73]:
weekends =(((trips["time_cat"] == "rush_m") | (trips["time_cat"] == "day time")) &  (trips["weekday"] >= 6))
df_5 = create_plotting_data(weekends)
plot_circles(df_5, "Weekend_daytime.html", 0.1)

In [86]:
SXSW_start_2014 = datetime.datetime(2014, 3, 7, 0, 0, 0)
SXSW_end_2014 = datetime.datetime(2014, 3, 14, 23, 59, 59)
SXSW_start_2015 = datetime.datetime(2015, 3, 13, 0, 0, 0)
SXSW_end_2015 = datetime.datetime(2015, 3, 23, 23, 59, 59)
SXSW_start_2016 = datetime.datetime(2016, 3, 11, 0, 0, 0)
SXSW_end_2016 = datetime.datetime(2016, 3, 20, 23, 59, 59)

In [94]:
SXSW_mask = ((trips_t_control["start_time"] > SXSW_start_2014) & (trips_t_control["start_time"] < SXSW_end_2014)) | \
                   ((trips_t_control["start_time"] > SXSW_start_2015) & (trips_t_control["start_time"] < SXSW_end_2015)) | \
                  ((trips_t_control["start_time"] > SXSW_start_2016) & (trips_t_control["start_time"] < SXSW_end_2016))

In [95]:
SXSW_mask =(((trips["time_cat"] == "rush_m") | (trips["time_cat"] == "day time")) &  SXSW_mask)
df_6 = create_plotting_data(SXSW_mask)
plot_circles(df_6, "SXSW.html", 0.5)

In [80]:
ACL_start_2014_W1 = datetime.datetime(2014, 10, 3, 0, 0, 0)
ACL_end_2014_W1 = datetime.datetime(2014, 10, 5, 23, 59, 59)
ACL_start_2014_W2 = datetime.datetime(2014, 10, 10, 0, 0, 0)
ACL_end_2014_W2 = datetime.datetime(2014, 10, 12, 23, 59, 59)
ACL_start_2015_W1 = datetime.datetime(2015, 10, 2, 0, 0, 0)
ACL_end_2015_W1 = datetime.datetime(2015, 10, 4, 23, 59, 59)
ACL_start_2015_W2 = datetime.datetime(2015, 10, 9, 0, 0, 0)
ACL_end_2015_W2 = datetime.datetime(2015, 10, 11, 23, 59, 59)
ACL_start_2016_W1 = datetime.datetime(2016, 9, 30, 0, 0, 0)
ACL_end_2016_W1 = datetime.datetime(2016, 10, 2, 23, 59, 59)
ACL_start_2016_W2 = datetime.datetime(2016, 10, 7, 0, 0, 0)
ACL_end_2016_W2 = datetime.datetime(2016, 10, 9, 23, 59, 59)

ACL_mask = ((trips_t_control["start_time"] > ACL_start_2014_W1) & (trips_t_control["start_time"] < ACL_end_2014_W1)) | \
                  ((trips_t_control["start_time"] > ACL_start_2014_W2) & (trips_t_control["start_time"] < ACL_end_2014_W2)) | \
                   ((trips_t_control["start_time"] > ACL_start_2015_W1) & (trips["start_time"] < ACL_end_2015_W1)) | \
                  (trips_t_control["start_time"] > ACL_start_2015_W2) & (trips_t_control["start_time"] < ACL_end_2015_W2) | \
                 ((trips_t_control["start_time"] > ACL_start_2016_W1) & (trips_t_control["start_time"] < ACL_end_2016_W1)) | \
                ((trips_t_control["start_time"] > ACL_start_2016_W2) & (trips_t_control["start_time"] < ACL_end_2016_W2))

In [97]:
#SXSW_mask =(((trips["time_cat"] == "rush_m") | (trips["time_cat"] == "day time")) &  SXSW_mask)
df_7 = create_plotting_data(ACL_mask)
plot_circles(df_7, "ACL.html", 0.1)

In [8]:
rush_m_heatmap_df = borrow_heatmap_df[borrow_heatmap_df["time_cat"] == "rush_m"]
rush_m_heatmap_weight = borrow_heatmap_df.groupby(["latitude", "longitude"])["location"].count()
rush_m_station_lat , rush_m_station_lon = zip(*rush_m_heatmap_weight.index.values)
#rush_m_heatmap_weight.values

rush_m_heatmap_weight_p = rush_m_heatmap_weight.values/ float(sum(rush_m_heatmap_weight.values))
rush_m_heatmap_weight_p

array([0.15154806, 0.03105054, 0.25002604, 0.05867912, 0.07602696,
       0.06152084, 0.09450553, 0.13605999, 0.00496928, 0.13561365])

In [52]:
gmap = gmplot.GoogleMapPlotter(30.28398,-97.73792, 13)


for i in range(0, len(rush_m_station_lat)):
    gmap.heatmap([rush_m_station_lat[i]] , 
                 [rush_m_station_lon[i]], 
                 radius = rush_m_heatmap_weight_p[i] * 300
                )


# Draw
gmap.draw("rush_hour_in_morning.html")

In [124]:
gmap = gmplot.GoogleMapPlotter(30.28398,-97.73792, 13)


for i in range(0, 5):
    gmap.circle(rush_m_station_lat[i] , 
                 rush_m_station_lon[i], 
                 radius = rush_m_heatmap_weight_p[i] * 300,
                color = "blue"

                )
    print(rush_m_station_lat[i], rush_m_station_lon[i],  rush_m_heatmap_weight_p[i] * 300)

# Draw
gmap.draw("rush_hour_in_morning.html")

(30.27041, -97.75046, 45.46441908559356)
(30.271929999999998, -97.73854, 9.315162245398955)
(30.2726, -97.74127, 75.0078109889456)
(30.273359999999997, -97.73805, 17.603737372234537)
(30.274659999999997, -97.77028, 22.80808772112538)


In [129]:
gmap = gmplot.GoogleMapPlotter(30.28398,-97.73792, 13)


for i in range(5):
        gmap.circle(circle_df["latitude_x"][i] , 
                 circle_df["longitude_x"][i], 
                 radius = circle_df["bikes_borrowed"][i],
                 color = "red"
                )
        print(circle_df["latitude_x"][i], circle_df["longitude_x"][i],  circle_df["bikes_borrowed"][i])

# Draw
gmap.draw("rush_hour_in_morning.html")

(30.26383, -97.72864, 8)
(30.27106, -97.74563, 5)
(30.27907, -97.73715, 5)
(30.26587, -97.76826, 1)
(30.268890000000003, -97.74525, 28)
